In [76]:
import json

import pandas as pd

ashley this is going to be very lazy code, I make no apologies

In [77]:
bar = pd.read_csv('/Users/user/Downloads/bar-classifications.csv')
spiral = pd.read_csv('/Users/user/Downloads/spiral-classifications.csv')
len(bar), len(spiral)

(32, 91)

In [92]:
bar.head()

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,513560862,not-logged-in-46da1827356d0fb76975,NaN,46da1827356d0fb76975,25255,Bar,23.31,2023-09-29 12:34:49 UTC,NaN,NaN,"{""source"":""api"",""session"":""6f976fd4ea38c92ba30...","[{""task"":""T0"",""task_label"":""Which algorithm be...","{""92122936"":{""retired"":null,""Filename"":""371107...",92122936
1,513560933,not-logged-in-b4d28815907add4b3731,NaN,b4d28815907add4b3731,25255,Bar,23.31,2023-09-29 12:35:22 UTC,NaN,NaN,"{""source"":""api"",""session"":""6f976fd4ea38c92ba30...","[{""task"":""T0"",""task_label"":""Which algorithm be...","{""92122948"":{""retired"":null,""Filename"":""441329...",92122948
2,513560949,not-logged-in-b4d28815907add4b3731,NaN,b4d28815907add4b3731,25255,Bar,23.31,2023-09-29 12:35:36 UTC,NaN,NaN,"{""source"":""api"",""session"":""6f976fd4ea38c92ba30...","[{""task"":""T0"",""task_label"":""Which algorithm be...","{""92122919"":{""retired"":null,""Filename"":""612183...",92122919
3,513560980,not-logged-in-b4d28815907add4b3731,NaN,b4d28815907add4b3731,25255,Bar,23.31,2023-09-29 12:36:07 UTC,NaN,NaN,"{""source"":""api"",""session"":""6f976fd4ea38c92ba30...","[{""task"":""T0"",""task_label"":""Which algorithm be...","{""92122939"":{""retired"":null,""Filename"":""322595...",92122939
4,513561133,not-logged-in-3a7e4858e4cab8b64e6e,NaN,3a7e4858e4cab8b64e6e,25255,Bar,23.31,2023-09-29 12:38:01 UTC,NaN,NaN,"{""source"":""api"",""session"":""6f976fd4ea38c92ba30...","[{""task"":""T0"",""task_label"":""Which algorithm be...","{""92122928"":{""retired"":null,""Filename"":""395590...",92122928


In [79]:
bar.iloc[0]['annotations']

'[{"task":"T0","task_label":"Which algorithm best identifies the bars?","value":"Top row"},{"task":"T1","task_label":"How did the best algorithm do?","value":"Perfectly"},{"task":"T2","task_label":"How did the other algorithm do?","value":"Mostly well (some minor errors)"}]'

In [80]:


quality_renamer = {
    'Perfectly': 'perfect',
    'Mostly well (some minor errors)': 'mostly',
    'Mostly well, some small errors': 'mostly',
    'Mostly poorly (some major errors)': 'poorly',
    'Mostly poorly, some major errors': 'poorly',
    'Totally failed': 'failed'
}

bar_namer = {
    'Top row': 'gz3d',
    'Bottom row': 'zoobot-3d'
}

spiral_namer = {
    'Top row': 'sparcfire',
    'Middle row': 'gz3d',
    'Bottom row': 'zoobot-3d'
}
    
    

def load_bar_annotations(annotation_str):
    annotations = json.loads(annotation_str)
    if len(annotations) == 1:
        assert annotations[0]['value'] == "I can't decide, they're so similar!"
        return {
            'best_algorithm': 'tie'
        }
    assert len(annotations) == 3, len(annotations)
    return {
        'best_algorithm': bar_namer[annotations[0]['value']],
        'best_quality': quality_renamer[annotations[1]['value']],
        'second_quality': quality_renamer[annotations[2]['value']],
    }

def load_spiral_annotations(annotation_str):
    annotations = json.loads(annotation_str)
    assert len(annotations) == 4, len(annotations)
    return {
        'best_algorithm': spiral_namer[annotations[0]['value']],
        'best_quality': quality_renamer[annotations[1]['value']],
        'second_algorithm': spiral_namer[annotations[2]['value']],
        'second_quality': quality_renamer[annotations[3]['value']],
    }

In [81]:
load_bar_annotations(bar['annotations'].iloc[0])

{'best_algorithm': 'gz3d',
 'best_quality': 'perfect',
 'second_quality': 'mostly'}

In [82]:
load_spiral_annotations(spiral['annotations'].iloc[0])

{'best_algorithm': 'zoobot-3d',
 'best_quality': 'mostly',
 'second_algorithm': 'gz3d',
 'second_quality': 'mostly'}

In [83]:
bar_df = pd.DataFrame(data=bar['annotations'].apply(load_bar_annotations).tolist())
bar_df

,best_algorithm,best_quality,second_quality
0,gz3d,perfect,mostly
1,gz3d,poorly,poorly
2,gz3d,perfect,mostly
3,zoobot-3d,perfect,mostly
4,zoobot-3d,mostly,mostly
5,zoobot-3d,mostly,poorly
6,zoobot-3d,mostly,poorly
7,zoobot-3d,perfect,mostly
8,zoobot-3d,perfect,mostly
9,zoobot-3d,perfect,mostly


In [84]:
bar_df['best_algorithm'].value_counts()

zoobot-3d    17
gz3d          9
tie           6
Name: best_algorithm, dtype: int64

In [85]:
zoobot_quality_when_best = bar_df.query('best_algorithm == "zoobot-3d"')['best_quality'].value_counts()
zoobot_quality_when_worst = bar_df.query('best_algorithm == "gz3d"')['second_quality'].value_counts()
zoobot_quality_when_worst['perfect'] = zoobot_quality_when_worst.get('perfect', 0)

zoobot_quality_when_best + zoobot_quality_when_worst

mostly     14
perfect     8
poorly      4
dtype: int64

In [86]:
gz3d_quality_when_best = bar_df.query('best_algorithm == "gz3d"')['best_quality'].value_counts()
gz3d_quality_when_worst = bar_df.query('best_algorithm == "zoobot-3d"')['second_quality'].value_counts()
gz3d_quality_when_worst['perfect'] = gz3d_quality_when_worst.get('perfect', 0)

gz3d_quality_when_best + gz3d_quality_when_worst

mostly     14
perfect     4
poorly      8
dtype: int64

## repeat for spirals

In [87]:
spiral_df = pd.DataFrame(data=spiral['annotations'].apply(load_spiral_annotations).tolist())
spiral_df

,best_algorithm,best_quality,second_algorithm,second_quality
0,zoobot-3d,mostly,gz3d,mostly
1,gz3d,mostly,zoobot-3d,mostly
2,zoobot-3d,mostly,gz3d,mostly
3,zoobot-3d,perfect,gz3d,mostly
4,zoobot-3d,perfect,gz3d,mostly
...,...,...,...,...
86,zoobot-3d,perfect,gz3d,mostly
87,zoobot-3d,perfect,gz3d,perfect
88,zoobot-3d,perfect,gz3d,mostly
89,zoobot-3d,perfect,gz3d,mostly


In [88]:
spiral_df['best_algorithm'].value_counts()

zoobot-3d    74
gz3d         17
Name: best_algorithm, dtype: int64

In [89]:
spiral_df['second_algorithm'].value_counts()

gz3d         73
zoobot-3d    17
sparcfire     1
Name: second_algorithm, dtype: int64

In [90]:
zoobot_quality_when_best = spiral_df.query('best_algorithm == "zoobot-3d"')['best_quality'].value_counts()
zoobot_quality_when_middle = spiral_df.query('second_algorithm == "zoobot-3d"')['second_quality'].value_counts()
# zoobot_quality_when_worst = spiral_df.query('best_algorithm == "gz3d"')['second_quality'].value_counts()  # actually didn't measure
zoobot_quality_when_middle['perfect'] = zoobot_quality_when_middle.get('perfect', 0)

zoobot_quality_when_best + zoobot_quality_when_worst

mostly     49
perfect    30
poorly      4
dtype: int64

In [91]:
gz3d_quality_when_best = spiral_df.query('best_algorithm == "gz3d"')['best_quality'].value_counts()
gz3d_quality_when_middle = spiral_df.query('second_algorithm == "gz3d"')['second_quality'].value_counts()
# gz3d_quality_when_worst = spiral_df.query('best_algorithm == "gz3d"')['second_quality'].value_counts()  # actually didn't measure
gz3d_quality_when_middle['perfect'] = gz3d_quality_when_middle.get('perfect', 0)

gz3d_quality_when_best + gz3d_quality_when_worst

mostly     23
perfect     3
poorly      8
dtype: int64